In [6]:
try:
    h2o.remove_all()
    h2o.shutdown()
except:
    pass

[WARNING] in <ipython-input-6-572c341a48bb> line 3:
    >>> h2o.shutdown()
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
H2O session _sid_9eb7 closed.


In [4]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
import h2o
import numpy as np
import pandas as pd
from h2o.estimators.word2vec import H2OWord2vecEstimator
pd.options.display.max_columns= None
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
from h2o.automl import H2OAutoML


In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/m/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplripzmu0
  JVM stdout: /tmp/tmplripzmu0/h2o_m_started_from_python.out
  JVM stderr: /tmp/tmplripzmu0/h2o_m_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,2 months and 18 days
H2O cluster name:,H2O_from_python_m_ieq5h6
H2O cluster total nodes:,1
H2O cluster free memory:,3.883 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [9]:
gl=h2o.import_file('glove.840B.300d.txt')



Parse progress: |█████████ (failed)


OSError: Job with key $03017f00000132d4ffffffff$_b2ebf199e3872a36aa325ec422fe4397 failed with an exception: DistributedException from /127.0.0.1:54321: 'Java heap space', caused by java.lang.OutOfMemoryError: Java heap space
stacktrace: 
DistributedException from /127.0.0.1:54321: 'Java heap space', caused by java.lang.OutOfMemoryError: Java heap space
	at water.MRTask.getResult(MRTask.java:479)
	at water.MRTask.getResult(MRTask.java:487)
	at water.MRTask.doAll(MRTask.java:403)
	at water.parser.ParseDataset.parseAllKeys(ParseDataset.java:257)
	at water.parser.ParseDataset.access$000(ParseDataset.java:29)
	at water.parser.ParseDataset$ParserFJTask.compute2(ParseDataset.java:206)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1417)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:974)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1477)
	at jsr166y.For